In [1]:
import torch
import torch.optim as optim
import numpy as np
from NGCF import NGCF
from utility.helper import *
# from utility.batch_test import *
from utility.load_data import *
import multiprocessing
import heapq
import utility.metrics as metrics

import warnings
warnings.filterwarnings('ignore')
from time import time
import easydict

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
torch.__version__

'2.1.0+cu121'

In [3]:
args = easydict.EasyDict({
    'dataset': 'evdriver',
    'regs' : '[1e-5]',
    'embed_size': 64,
    'layer_size': '[64,64,64]',
    'lr': 0.0001,
    'save_flag': 1,
    'pretrain': 0,
    'batch_size': 1024,
    'epoch': 100,
    'verbose': 1,
    'node_dropout': [0.1],
    'mess_dropout': [0.1,0.1,0.1],
    'gpu_id': 0,
    'weights_path': './models',
    'Ks': '[20, 40, 60, 80, 100]',
    'test_flag':'part'})

Ks = eval(args.Ks)
cores = multiprocessing.cpu_count() // 2

In [28]:
cores

10

In [4]:
def ranklist_by_heapq(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = 0.
    return r, auc

def get_auc(item_score, user_pos_test):
    item_score = sorted(item_score.items(), key=lambda kv: kv[1])
    item_score.reverse()
    item_sort = [x[0] for x in item_score]
    posterior = [x[1] for x in item_score]

    r = []
    for i in item_sort:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = metrics.auc(ground_truth=r, prediction=posterior)
    return auc

def ranklist_by_sorted(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = get_auc(item_score, user_pos_test)
    return r, auc

def get_performance(user_pos_test, r, auc, Ks):
    precision, recall, ndcg, hit_ratio = [], [], [], []

    for K in Ks:
        precision.append(metrics.precision_at_k(r, K))
        recall.append(metrics.recall_at_k(r, K, len(user_pos_test)))
        ndcg.append(metrics.ndcg_at_k(r, K, user_pos_test))
        hit_ratio.append(metrics.hit_at_k(r, K))

    return {'recall': np.array(recall), 'precision': np.array(precision),
            'ndcg': np.array(ndcg), 'hit_ratio': np.array(hit_ratio), 'auc': auc}

def test_one_user(x):
    # user u's ratings for user u
    rating = x[0]
    #uid
    u = x[1]
    #user u's items in the training set
    try:
        training_items = data_generator.train_items[u]
    except Exception:
        training_items = []
    #user u's items in the test set
    user_pos_test = data_generator.test_set[u]

    all_items = set(range(ITEM_NUM))

    test_items = list(all_items - set(training_items))

    if args.test_flag == 'part':
        r, auc = ranklist_by_heapq(user_pos_test, test_items, rating, Ks)
    else:
        r, auc = ranklist_by_sorted(user_pos_test, test_items, rating, Ks)

    return get_performance(user_pos_test, r, auc, Ks)

def test(model, users_to_test, drop_flag=False, batch_test_flag=False):
    result = {'precision': np.zeros(len(Ks)), 'recall': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)),
              'hit_ratio': np.zeros(len(Ks)), 'auc': 0.}

    pool = multiprocessing.Pool(cores)

    u_batch_size = BATCH_SIZE * 2
    i_batch_size = BATCH_SIZE

    test_users = users_to_test
    n_test_users = len(test_users)
    n_user_batchs = n_test_users // u_batch_size + 1

    count = 0

    for u_batch_id in range(n_user_batchs):
        start = u_batch_id * u_batch_size
        end = (u_batch_id + 1) * u_batch_size

        user_batch = test_users[start: end]

        if batch_test_flag:
            # batch-item test
            n_item_batchs = ITEM_NUM // i_batch_size + 1
            rate_batch = np.zeros(shape=(len(user_batch), ITEM_NUM))

            i_count = 0
            for i_batch_id in range(n_item_batchs):
                i_start = i_batch_id * i_batch_size
                i_end = min((i_batch_id + 1) * i_batch_size, ITEM_NUM)

                item_batch = range(i_start, i_end)

                if drop_flag == False:
                    u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                                  item_batch,
                                                                  [],
                                                                  drop_flag=False)
                    i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
                else:
                    u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                                  item_batch,
                                                                  [],
                                                                  drop_flag=True)
                    i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

                rate_batch[:, i_start: i_end] = i_rate_batch
                i_count += i_rate_batch.shape[1]

            assert i_count == ITEM_NUM

        else:
            # all-item test
            item_batch = range(ITEM_NUM)

            if drop_flag == False:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                              item_batch,
                                                              [],
                                                              drop_flag=False)
                rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
            else:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                              item_batch,
                                                              [],
                                                              drop_flag=True)
                rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

        user_batch_rating_uid = zip(rate_batch.numpy(), user_batch)
        batch_result = pool.map(test_one_user, user_batch_rating_uid)
        count += len(batch_result)

        for re in batch_result:
            result['precision'] += re['precision']/n_test_users
            result['recall'] += re['recall']/n_test_users
            result['ndcg'] += re['ndcg']/n_test_users
            result['hit_ratio'] += re['hit_ratio']/n_test_users
            result['auc'] += re['auc']/n_test_users


    assert count == n_test_users
    pool.close()
    return result


In [5]:
data_generator = Data(path='../Data/evdriver', batch_size=1024)
USR_NUM, ITEM_NUM = data_generator.n_users, data_generator.n_items
N_TRAIN, N_TEST = data_generator.n_train, data_generator.n_test
BATCH_SIZE = args.batch_size

n_users=185336, n_items=30895
n_interactions=852546
n_train=667210, n_test=185336, sparsity=0.00015


In [6]:
plain_adj, norm_adj, mean_adj = data_generator.get_adj_mat()

already load adj matrix (216231, 216231) 0.35448646545410156


In [7]:
users_to_test = list(data_generator.test_set.keys())

In [8]:
users_to_test[-1]

185335

In [9]:
import pickle
torch.manual_seed(34)
np.random.seed(34)

drop_flag = False
batch_test_flag=False

args.device = torch.device('cuda:' + str(args.gpu_id))
model = NGCF(data_generator.n_users,
                 data_generator.n_items,
                 norm_adj, args).to(args.device)

In [10]:
with open('./models359.pkl', 'rb') as f:
    model.load_state_dict(torch.load(f))

In [ ]:
result = {'precision': np.zeros(len(Ks)), 'recall': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)),
            'hit_ratio': np.zeros(len(Ks)), 'auc': 0.}

# pool = multiprocessing.Pool(cores)

u_batch_size = BATCH_SIZE * 2
i_batch_size = BATCH_SIZE

test_users = users_to_test
n_test_users = len(test_users)
n_user_batchs = n_test_users // u_batch_size + 1
print(n_user_batchs)
count = 0

entire_rank = torch.tensor([])

for u_batch_id in tqdm(range(n_user_batchs)):
    start = u_batch_id * u_batch_size
    end = (u_batch_id + 1) * u_batch_size

    user_batch = test_users[start: end]

    if batch_test_flag:
        # batch-item test
        n_item_batchs = ITEM_NUM // i_batch_size + 1
        rate_batch = np.zeros(shape=(len(user_batch), ITEM_NUM))

        i_count = 0
        for i_batch_id in range(n_item_batchs):
            i_start = i_batch_id * i_batch_size
            i_end = min((i_batch_id + 1) * i_batch_size, ITEM_NUM)

            item_batch = range(i_start, i_end)

            if drop_flag == False:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                                item_batch,
                                                                [],
                                                                drop_flag=False)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
            else:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                                item_batch,
                                                                [],
                                                                drop_flag=True)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

            rate_batch[:, i_start: i_end] = i_rate_batch
            i_count += i_rate_batch.shape[1]

        assert i_count == ITEM_NUM

    else:
        # all-item test
        item_batch = range(ITEM_NUM)

        if drop_flag == False:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                            item_batch,
                                                            [],
                                                            drop_flag=False)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
        else:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                            item_batch,
                                                            [],
                                                            drop_flag=True)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

    
    entire_rank = torch.cat((entire_rank, rate_batch), dim=0)
    user_batch_rating_uid = zip(rate_batch.numpy(), user_batch)
#     batch_result = pool.map(test_one_user, user_batch_rating_uid)
    batch_result = []
    for user_rating in user_batch_rating_uid:
        result = test_one_user(user_rating)
        batch_result.append(result)

    count += len(batch_result)

    for re in batch_result:
        result['precision'] += re['precision']/n_test_users
        result['recall'] += re['recall']/n_test_users
        result['ndcg'] += re['ndcg']/n_test_users
        result['hit_ratio'] += re['hit_ratio']/n_test_users
        result['auc'] += re['auc']/n_test_users


assert count == n_test_users
# pool.close()

91


 65%|██████▍   | 59/91 [20:53<11:45, 22.05s/it]

In [ ]:
import pickle
import torch
import numpy as np
from tqdm import tqdm

torch.manual_seed(34)
np.random.seed(34)

drop_flag = False
batch_test_flag = False

args.device = torch.device('cuda:' + str(args.gpu_id))
model = NGCF(data_generator.n_users, data_generator.n_items, norm_adj, args).to(args.device)

with open('./models359.pkl', 'rb') as f:
    model.load_state_dict(torch.load(f))

result = {'precision': np.zeros(len(Ks)), 'recall': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)),
          'hit_ratio': np.zeros(len(Ks)), 'auc': 0.}

u_batch_size = BATCH_SIZE * 2
i_batch_size = BATCH_SIZE

test_users = users_to_test
n_test_users = len(test_users)
n_user_batches = n_test_users // u_batch_size + 1
print(n_user_batches)
count = 0

entire_rank = torch.tensor([])

# 중간 결과 저장 경로 설정
checkpoint_path = './intermediate_results.pkl'

for u_batch_id in tqdm(range(n_user_batches)):
    start = u_batch_id * u_batch_size
    end = (u_batch_id + 1) * u_batch_size

    user_batch = test_users[start: end]

    if batch_test_flag:
        n_item_batches = ITEM_NUM // i_batch_size + 1
        rate_batch = np.zeros(shape=(len(user_batch), ITEM_NUM))

        i_count = 0
        for i_batch_id in range(n_item_batches):
            i_start = i_batch_id * i_batch_size
            i_end = min((i_batch_id + 1) * i_batch_size, ITEM_NUM)

            item_batch = range(i_start, i_end)

            if drop_flag == False:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
            else:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

            rate_batch[:, i_start: i_end] = i_rate_batch
            i_count += i_rate_batch.shape[1]

        assert i_count == ITEM_NUM

    else:
        item_batch = range(ITEM_NUM)

        if drop_flag == False:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
        else:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

    entire_rank = torch.cat((entire_rank, rate_batch), dim=0)
    user_batch_rating_uid = zip(rate_batch.numpy(), user_batch)
    
    batch_result = []
    for user_rating in user_batch_rating_uid:
        result = test_one_user(user_rating)
        batch_result.append(result)

    count += len(batch_result)

    for re in batch_result:
        result['precision'] += re['precision'] / n_test_users
        result['recall'] += re['recall'] / n_test_users
        result['ndcg'] += re['ndcg'] / n_test_users
        result['hit_ratio'] += re['hit_ratio'] / n_test_users
        result['auc'] += re['auc'] / n_test_users

    # 메모리 해제
    del rate_batch, user_batch_rating_uid, batch_result
    torch.cuda.empty_cache()
    
    # 주기적으로 중간 결과 저장
    if (u_batch_id + 1) % 10 == 0 or (u_batch_id + 1) == n_user_batches:
        with open(checkpoint_path, 'wb') as f:
            pickle.dump((result, entire_rank), f)
        print(f"Intermediate results saved at batch {u_batch_id + 1}")

assert count == n_test_users

# 최종 결과 저장
with open(checkpoint_path, 'wb') as f:
    pickle.dump((result, entire_rank), f)
print("Final results saved.")


91


 11%|█         | 10/91 [03:32<29:19, 21.72s/it]

Intermediate results saved at batch 10


 22%|██▏       | 20/91 [07:29<34:41, 29.32s/it]

Intermediate results saved at batch 20


...

 44%|████▍     | 40/91 [16:26<34:55, 41.08s/it]

Intermediate results saved at batch 40


 55%|█████▍    | 50/91 [21:30<32:57, 48.23s/it]

Intermediate results saved at batch 50


 65%|██████▍   | 59/91 [25:35<15:13, 28.54s/it]

In [9]:
import pickle
import torch
import numpy as np
from tqdm import tqdm

torch.manual_seed(34)
np.random.seed(34)

drop_flag = False
batch_test_flag = False

args.device = torch.device('cuda:' + str(args.gpu_id))
model = NGCF(data_generator.n_users, data_generator.n_items, norm_adj, args).to(args.device)

with open('./models359.pkl', 'rb') as f:
    model.load_state_dict(torch.load(f))

result = {'precision': np.zeros(len(Ks)), 'recall': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)),
          'hit_ratio': np.zeros(len(Ks)), 'auc': 0.}

u_batch_size = BATCH_SIZE * 2
i_batch_size = BATCH_SIZE

test_users = users_to_test
n_test_users = len(test_users)
n_user_batches = n_test_users // u_batch_size + 1
print(n_user_batches)
count = 0

# 중간 결과 저장 경로 설정
checkpoint_path = './intermediate_results.pkl'

def save_intermediate_results(result, batch_id, path=checkpoint_path):
    with open(path, 'wb') as f:
        pickle.dump(result, f)
    print(f"Intermediate results saved at batch {batch_id}")

for u_batch_id in tqdm(range(n_user_batches)):
    start = u_batch_id * u_batch_size
    end = (u_batch_id + 1) * u_batch_size

    user_batch = test_users[start: end]

    if batch_test_flag:
        n_item_batches = ITEM_NUM // i_batch_size + 1
        rate_batch = np.zeros(shape=(len(user_batch), ITEM_NUM))

        i_count = 0
        for i_batch_id in range(n_item_batches):
            i_start = i_batch_id * i_batch_size
            i_end = min((i_batch_id + 1) * i_batch_size, ITEM_NUM)

            item_batch = range(i_start, i_end)

            if drop_flag == False:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
            else:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

            rate_batch[:, i_start: i_end] = i_rate_batch
            i_count += i_rate_batch.shape[1]

        assert i_count == ITEM_NUM

    else:
        item_batch = range(ITEM_NUM)

        if drop_flag == False:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
        else:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

    user_batch_rating_uid = zip(rate_batch.numpy(), user_batch)
    
    batch_result = []
    for user_rating in user_batch_rating_uid:
        user_result = test_one_user(user_rating)
        batch_result.append(user_result)

    count += len(batch_result)

    for re in batch_result:
        result['precision'] += re['precision'] / n_test_users
        result['recall'] += re['recall'] / n_test_users
        result['ndcg'] += re['ndcg'] / n_test_users
        result['hit_ratio'] += re['hit_ratio'] / n_test_users
        result['auc'] += re['auc'] / n_test_users

    # 메모리 해제
    del rate_batch, user_batch_rating_uid, batch_result
    torch.cuda.empty_cache()
    
    # 주기적으로 중간 결과 저장
    if (u_batch_id + 1) % 5 == 0 or (u_batch_id + 1) == n_user_batches:
        save_intermediate_results(result, u_batch_id + 1)

assert count == n_test_users

# 최종 결과 저장
save_intermediate_results(result, 'final')
print("Final results saved.")


91


  5%|▌         | 5/91 [01:43<29:24, 20.51s/it]

Intermediate results saved at batch 5


 11%|█         | 10/91 [03:25<27:28, 20.35s/it]

Intermediate results saved at batch 10


...

Intermediate results saved at batch 85


 99%|█████████▉| 90/91 [30:30<00:20, 20.35s/it]

Intermediate results saved at batch 90


100%|██████████| 91/91 [30:40<00:00, 20.22s/it]

Intermediate results saved at batch 91
Intermediate results saved at batch final
Final results saved.


In [10]:
import numpy as np
import torch
from tqdm import tqdm

# Set seeds for reproducibility
torch.manual_seed(34)
np.random.seed(34)

# Initialize flags
drop_flag = False
batch_test_flag = False

# Define the device and model
args.device = torch.device('cuda:' + str(args.gpu_id))
model = NGCF(data_generator.n_users, data_generator.n_items, norm_adj, args).to(args.device)

# Load model weights
with open('./models359.pkl', 'rb') as f:
    model.load_state_dict(torch.load(f))

# Define result dictionary
result = {'precision': np.zeros(len(Ks)), 'recall': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)),
          'hit_ratio': np.zeros(len(Ks)), 'auc': 0.}

# Batch sizes
u_batch_size = BATCH_SIZE * 2
i_batch_size = BATCH_SIZE

# Test users
test_users = users_to_test
n_test_users = len(test_users)
n_user_batches = n_test_users // u_batch_size + 1
print(n_user_batches)
count = 0

# Initialize entire rank tensor
entire_rank = []

# Iterate over user batches
for u_batch_id in tqdm(range(n_user_batches)):
    start = u_batch_id * u_batch_size
    end = (u_batch_id + 1) * u_batch_size

    user_batch = test_users[start: end]

    if batch_test_flag:
        # Batch-item test
        n_item_batches = ITEM_NUM // i_batch_size + 1
        rate_batch = np.zeros(shape=(len(user_batch), ITEM_NUM))

        i_count = 0
        for i_batch_id in range(n_item_batches):
            i_start = i_batch_id * i_batch_size
            i_end = min((i_batch_id + 1) * i_batch_size, ITEM_NUM)

            item_batch = range(i_start, i_end)

            if not drop_flag:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()
            else:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()

            rate_batch[:, i_start: i_end] = i_rate_batch
            i_count += i_rate_batch.shape[1]

        assert i_count == ITEM_NUM

    else:
        # All-item test
        item_batch = range(ITEM_NUM)

        if not drop_flag:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()
        else:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()

    # Append results to entire_rank list
    entire_rank.append(rate_batch)

    user_batch_rating_uid = zip(rate_batch, user_batch)

    # Simulate multiprocessing pool.map
    batch_result = []
    for user_rating in user_batch_rating_uid:
        re = test_one_user(user_rating)
        batch_result.append(re)

    count += len(batch_result)

    for re in batch_result:
        result['precision'] += re['precision'] / n_test_users
        result['recall'] += re['recall'] / n_test_users
        result['ndcg'] += re['ndcg'] / n_test_users
        result['hit_ratio'] += re['hit_ratio'] / n_test_users
        result['auc'] += re['auc'] / n_test_users

assert count == n_test_users

# Save the entire_rank as a .npz file
np.savez_compressed('entire_rank.npz', *entire_rank)

91


100%|██████████| 91/91 [30:38<00:00, 20.20s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
import numpy as np
import torch
import os
from tqdm import tqdm

# Save the entire_rank in smaller chunks
def save_batches(entire_rank, batch_size=1024, save_dir='batches'):
    os.makedirs(save_dir, exist_ok=True)
    num_batches = len(entire_rank) // batch_size + (1 if len(entire_rank) % batch_size != 0 else 0)
    for i in range(num_batches):
        start = i * batch_size
        end = (i + 1) * batch_size
        np.savez_compressed(f'{save_dir}/batch_{i}.npz', *entire_rank[start:end])

# Load batches one by one and process them
def load_batches(batch_dir='batches'):
    batch_files = sorted([os.path.join(batch_dir, f) for f in os.listdir(batch_dir) if f.endswith('.npz')])
    entire_rank = []
    for batch_file in tqdm(batch_files):
        loaded_batch = np.load(batch_file)
        batch_data = [loaded_batch[key] for key in loaded_batch]
        entire_rank.append(np.concatenate(batch_data, axis=0))
    return torch.tensor(np.concatenate(entire_rank, axis=0))

# Example usage
torch.manual_seed(34)
np.random.seed(34)

drop_flag = False
batch_test_flag = False

args.device = torch.device('cuda:' + str(args.gpu_id))
model = NGCF(data_generator.n_users, data_generator.n_items, norm_adj, args).to(args.device)

with open('./models359.pkl', 'rb') as f:
    model.load_state_dict(torch.load(f))

result = {'precision': np.zeros(len(Ks)), 'recall': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)),
          'hit_ratio': np.zeros(len(Ks)), 'auc': 0.}

u_batch_size = BATCH_SIZE * 2
i_batch_size = BATCH_SIZE

test_users = users_to_test
n_test_users = len(test_users)
n_user_batches = n_test_users // u_batch_size + 1
print(n_user_batches)
count = 0

entire_rank = []

for u_batch_id in tqdm(range(n_user_batches)):
    start = u_batch_id * u_batch_size
    end = (u_batch_id + 1) * u_batch_size

    user_batch = test_users[start:end]

    if batch_test_flag:
        n_item_batches = ITEM_NUM // i_batch_size + 1
        rate_batch = np.zeros(shape=(len(user_batch), ITEM_NUM))

        i_count = 0
        for i_batch_id in range(n_item_batches):
            i_start = i_batch_id * i_batch_size
            i_end = min((i_batch_id + 1) * i_batch_size, ITEM_NUM)

            item_batch = range(i_start, i_end)

            if not drop_flag:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()
            else:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()

            rate_batch[:, i_start:i_end] = i_rate_batch
            i_count += i_rate_batch.shape[1]

        assert i_count == ITEM_NUM

    else:
        item_batch = range(ITEM_NUM)

        if not drop_flag:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()
        else:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()

    entire_rank.append(rate_batch)
    user_batch_rating_uid = zip(rate_batch, user_batch)

    batch_result = []
    for user_rating in user_batch_rating_uid:
        re = test_one_user(user_rating)
        batch_result.append(re)

    count += len(batch_result)

    for re in batch_result:
        result['precision'] += re['precision'] / n_test_users
        result['recall'] += re['recall'] / n_test_users
        result['ndcg'] += re['ndcg'] / n_test_users
        result['hit_ratio'] += re['hit_ratio'] / n_test_users
        result['auc'] += re['auc'] / n_test_users

assert count == n_test_users

# Save the entire_rank in batches
save_batches(entire_rank)

# Load the entire_rank from batches
entire_rank = load_batches()


91


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
import numpy as np
import torch
from tqdm import tqdm
import os

# Set seeds for reproducibility
torch.manual_seed(34)
np.random.seed(34)

# Initialize flags
drop_flag = False
batch_test_flag = False

# Define the device and model
args.device = torch.device('cuda:' + str(args.gpu_id))
model = NGCF(data_generator.n_users, data_generator.n_items, norm_adj, args).to(args.device)

# Load model weights
with open('./models359.pkl', 'rb') as f:
    model.load_state_dict(torch.load(f))

# Define result dictionary
result = {'precision': np.zeros(len(Ks)), 'recall': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)),
          'hit_ratio': np.zeros(len(Ks)), 'auc': 0.}

# Batch sizes
u_batch_size = BATCH_SIZE * 2
i_batch_size = BATCH_SIZE

# Test users
test_users = users_to_test
n_test_users = len(test_users)
n_user_batches = n_test_users // u_batch_size + 1
print(n_user_batches)
count = 0

# Initialize entire rank list
entire_rank = []
batch_count = 0
save_count = 0

# Directory to save batches
output_dir = 'batches'
os.makedirs(output_dir, exist_ok=True)

# Iterate over user batches
for u_batch_id in tqdm(range(n_user_batches)):
    start = u_batch_id * u_batch_size
    end = (u_batch_id + 1) * u_batch_size

    user_batch = test_users[start: end]

    if batch_test_flag:
        # Batch-item test
        n_item_batches = ITEM_NUM // i_batch_size + 1
        rate_batch = np.zeros(shape=(len(user_batch), ITEM_NUM))

        i_count = 0
        for i_batch_id in range(n_item_batches):
            i_start = i_batch_id * i_batch_size
            i_end = min((i_batch_id + 1) * i_batch_size, ITEM_NUM)

            item_batch = range(i_start, i_end)

            if not drop_flag:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()
            else:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
                i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()

            rate_batch[:, i_start: i_end] = i_rate_batch
            i_count += i_rate_batch.shape[1]

        assert i_count == ITEM_NUM

    else:
        # All-item test
        item_batch = range(ITEM_NUM)

        if not drop_flag:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=False)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()
        else:
            u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch, item_batch, [], drop_flag=True)
            rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu().numpy()

    # Append results to entire_rank list
    entire_rank.append(rate_batch)
    batch_count += 1

    # Save every 5 batches
    if batch_count == 5:
        save_path = os.path.join(output_dir, f'entire_rank_batch_{save_count}.npy')
        np.save(save_path, np.array(entire_rank))
        entire_rank = []  # Reset the list
        batch_count = 0
        save_count += 1

    user_batch_rating_uid = zip(rate_batch, user_batch)

    # Simulate multiprocessing pool.map
    batch_result = []
    for user_rating in user_batch_rating_uid:
        re = test_one_user(user_rating)
        batch_result.append(re)

    count += len(batch_result)

    for re in batch_result:
        result['precision'] += re['precision'] / n_test_users
        result['recall'] += re['recall'] / n_test_users
        result['ndcg'] += re['ndcg'] / n_test_users
        result['hit_ratio'] += re['hit_ratio'] / n_test_users
        result['auc'] += re['auc'] / n_test_users

assert count == n_test_users

# Save remaining batches if any
if entire_rank:
    save_path = os.path.join(output_dir, f'entire_rank_batch_{save_count}.npy')
    np.save(save_path, np.array(entire_rank))

print("All batches saved successfully.")

91


100%|██████████| 91/91 [30:44<00:00, 20.27s/it]

All batches saved successfully.


In [ ]:
# Load the entire_rank from the .npz file
loaded_entire_rank = np.load('entire_rank.npz')
entire_rank = [loaded_entire_rank[key] for key in loaded_entire_rank]

# Convert loaded NumPy arrays back to torch tensors
entire_rank = torch.tensor(np.concatenate(entire_rank, axis=0))

In [19]:
loaded_entire_rank = np.load('./batches/entire_rank_batch_0.npy')

In [20]:
entire_rank = [loaded_entire_rank[key] for key in loaded_entire_rank]

IndexError: arrays used as indices must be of integer (or boolean) type

In [11]:
import pickle, pickle5
import pandas as pd

with open('../Data/evdriver/preprocessed/meta.pickle', 'rb') as f:
    meta = pickle.load(f)

with open('../Data/evdriver/preprocessed/drivers.pickle', 'rb') as f:
    train_dv = pickle.load(f)

with open('../Data/evdriver/preprocessed/testset_df.pickle', 'rb') as f:
    test_dv = pickle.load(f)

test_dv = test_dv.sort_values(by=['Driver']).reset_index(drop=True)


In [12]:
def evaluation(result, driver_id, test_dv, meta, criteria):
    location = test_dv.loc[test_dv['Driver'] == driver_id][criteria].values[0]
    teststat = test_dv.loc[test_dv['Driver'] == driver_id]['statid'].values[0]
    speed = test_dv.loc[test_dv['Driver'] == driver_id]['speed'].values
    if len(speed) == 1:
        flag = True
    else:
        flag = False

    driver_pref = result[driver_id]
    #[1011, 14000, 13000] -> statid의 list
    driver_rank = driver_pref.argsort()[::-1].astype(str)
    #하나의 Cluster로 위치 필터링
    driver_meta = meta[meta[criteria] == location]
    recommended_df = pd.DataFrame([])

    for i in range(len(driver_rank)):
        if len(recommended_df) > 19:
            false_count = len(recommended_df) - recommended_df.duplicated('statid').sum()
            if false_count == 20:
                break
        #추천된 statid가 Cluster에 속하면,
        if driver_rank[i] in driver_meta['statid'].values:
            stat_driver = driver_meta.loc[driver_meta['statid'] == driver_rank[i]]
            #추천된 statid의 speed가 testset의 speed와 같으면,
            if flag:
                if stat_driver['speed'].values[0] == speed:
                    recommended_df = pd.concat([recommended_df, stat_driver])
            else:
                recommended_df = pd.concat([recommended_df, stat_driver])
                
    if flag:
        recommended_df = recommended_df.loc[recommended_df['speed'] == speed[0]].reset_index(drop=True)
    else:
        recommended_df = recommended_df.drop_duplicates('statid').reset_index(drop=True)

    try:
        rank = recommended_df.loc[recommended_df['statid'] == teststat].index[0] 
        rank = rank + 1
    except:
        rank = 0
    return recommended_df, rank, len(recommended_df)

def get_coordinates(driver_id, meta, test_dv, candidate):
    gt_zscode = test_dv.loc[test_dv['Driver'] == driver_id]['zscode'].values[0]
    gt_cluster = test_dv.loc[test_dv['Driver'] == driver_id]['Cluster'].values[0]
    gt_speeds = test_dv.loc[test_dv['Driver'] == driver_id]['speed']
    if len(gt_speeds.values) == 1:
        gt_speed = gt_speeds.values[0]
    else:
        gt_speed = gt_speeds.value_counts().keys()[0]
    
    meta_zscode = meta[meta['zscode'] == gt_zscode]
    meta_zscode = meta_zscode.loc[meta_zscode['speed'] == gt_speed]
    meta_cluster = meta.loc[meta['Cluster'] == gt_cluster]
    meta_cluster = meta_cluster.loc[meta_cluster['speed'] == gt_speed]

    zscode_cor = meta_zscode[['lat','lng']].values
    cluster_cor = meta_cluster[['lat','lng']].values
    candidate_cor = candidate[['lat','lng']].values
    ground_truth_cor = test_dv.loc[test_dv['Driver'] == driver_id][['lat','lng']].values
    

    return zscode_cor, cluster_cor, candidate_cor, ground_truth_cor

def show_plot(zscode_cor, cluster_cor, candidate_cor, gt_cor, driver_id):
    plt.scatter(zscode_cor[:, 1], zscode_cor[:, 0], marker='o', color='blue', label='zscode', s=5)
    plt.scatter(cluster_cor[:, 1], cluster_cor[:, 0], marker='o', color='red', label='cluster', s=5)
    plt.scatter(candidate_cor[:, 1], candidate_cor[:, 0], marker='o', color='green', label='candidate', s=5)
    plt.scatter(gt_cor[:, 1], gt_cor[:, 0], marker='x', color='black', label='Ground Truth', s=30)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(f'Scatter Plot of Coordinates for driver: {driver_id}')
    plt.legend()
    plt.show()

def visulization(result, driver_id, test_dv, meta, criteria):
    recommended_df, rank, length = evaluation(result, driver_id, test_dv, meta, criteria)
    zscode_cor, cluster_cor, candidate_cor, gt_cor = get_coordinates(driver_id, meta, test_dv, recommended_df)
    show_plot(zscode_cor, cluster_cor, candidate_cor, gt_cor, driver_id)
    return 

In [22]:
test_dv['speed']

0         1
1         1
2         1
3         0
4         0
         ..
201275    1
201276    0
201277    0
201278    0
201279    1
Name: speed, Length: 201280, dtype: int64

In [14]:
visulization(entire_rank, 0, test_dv, meta, 'Cluster')

KeyError: 'speed'

In [ ]:
visulization(entire_rank, 19755, test_dv, meta, 'Cluster')

In [12]:
import pickle
import torch
import numpy as np
import random
import logging
import datetime
import os
from utility.parser import parse_args
from utility.batch_test import *
from utility.load_data import *
from model import *
from tqdm import tqdm
from time import time
from copy import deepcopy

args = parse_args()
seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def load_adjacency_list_data(adj_mat):
    tmp = adj_mat.tocoo()
    all_h_list = list(tmp.row)
    all_t_list = list(tmp.col)
    all_v_list = list(tmp.data)

    return all_h_list, all_t_list, all_v_list

if __name__ == '__main__':
    """
    *********************************************************
    Prepare the log file
    """
    curr_time = datetime.datetime.now()
    if not os.path.exists('log'):
        os.mkdir('log')
    logger = logging.getLogger('train_logger')
    logger.setLevel(logging.INFO)
    logfile = logging.FileHandler('log/{}.log'.format(args.dataset), 'a', encoding='utf-8')
    logfile.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(message)s')
    logfile.setFormatter(formatter)
    logger.addHandler(logfile)

    """
    *********************************************************
    Prepare the dataset
    """
    data_generator = Data(args)
    logger.info(data_generator.get_statistics())

    print("************************* Run with following settings 🏃 ***************************")
    print(args)
    logger.info(args)
    print("************************************************************************************")

    config = dict()
    config['n_users'] = data_generator.n_users
    config['n_items'] = data_generator.n_items

    """
    *********************************************************
    Generate the adj matrix
    """
    plain_adj = data_generator.get_adj_mat()
    all_h_list, all_t_list, all_v_list = load_adjacency_list_data(plain_adj)
    config['plain_adj'] = plain_adj
    config['all_h_list'] = all_h_list
    config['all_t_list'] = all_t_list

    """
    *********************************************************
    Prepare the model and start training
    """
    model = NGCF(data_generator.n_users,
                 data_generator.n_items,
                 norm_adj, args).to(args.device)
    with open('./models359.pkl', 'rb') as f:
        model.load_state_dict(torch.load(f))

    result = {'precision': np.zeros(len(Ks)), 'recall': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)),
              'hit_ratio': np.zeros(len(Ks)), 'auc': 0.}

    u_batch_size = BATCH_SIZE * 2
    i_batch_size = BATCH_SIZE

    test_users = users_to_test
    n_test_users = len(test_users)
    n_user_batchs = n_test_users // u_batch_size + 1
    print(n_user_batchs)
    count = 0

    entire_rank = torch.tensor([])

    for u_batch_id in tqdm(range(n_user_batchs)):
        start = u_batch_id * u_batch_size
        end = (u_batch_id + 1) * u_batch_size
        user_batch = test_users[start:end]

        if batch_test_flag:
            # batch-item test
            n_item_batchs = ITEM_NUM // i_batch_size + 1
            rate_batch = np.zeros(shape=(len(user_batch), ITEM_NUM))

            i_count = 0
            for i_batch_id in range(n_item_batchs):
                i_start = i_batch_id * i_batch_size
                i_end = min((i_batch_id + 1) * i_batch_size, ITEM_NUM)

                item_batch = range(i_start, i_end)

                if drop_flag == False:
                    u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                                  item_batch,
                                                                  [],
                                                                  drop_flag=False)
                    i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
                else:
                    u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                                  item_batch,
                                                                  [],
                                                                  drop_flag=True)
                    i_rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

                rate_batch[:, i_start: i_end] = i_rate_batch
                i_count += i_rate_batch.shape[1]

            assert i_count == ITEM_NUM

        else:
            # all-item test
            item_batch = range(ITEM_NUM)

            if drop_flag == False:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                              item_batch,
                                                              [],
                                                              drop_flag=False)
                rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
            else:
                u_g_embeddings, pos_i_g_embeddings, _ = model(user_batch,
                                                              item_batch,
                                                              [],
                                                              drop_flag=True)
                rate_batch = model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()

        entire_rank = torch.cat((entire_rank, rate_batch), dim=0)
        user_batch_rating_uid = zip(rate_batch.numpy(), user_batch)
        
        # 멀티프로세싱 대신 단일 프로세스로 평가
        batch_result = [test_one_user(x) for x in user_batch_rating_uid]
        count += len(batch_result)

        for re in batch_result:
            result['precision'] += re['precision'] / n_test_users
            result['recall'] += re['recall'] / n_test_users
            result['ndcg'] += re['ndcg'] / n_test_users
            result['hit_ratio'] += re['hit_ratio'] / n_test_users
            result['auc'] += re['auc'] / n_test_users

    assert count == n_test_users

    # 최종 결과 출력
    print(result)


usage: ipykernel_launcher.py [-h] [--weights_path [WEIGHTS_PATH]]
                             [--data_path [DATA_PATH]]
                             [--proj_path [PROJ_PATH]] [--dataset [DATASET]]
                             [--pretrain PRETRAIN] [--verbose VERBOSE]
                             [--epoch EPOCH] [--embed_size EMBED_SIZE]
                             [--layer_size [LAYER_SIZE]]
                             [--batch_size BATCH_SIZE] [--regs [REGS]]
                             [--lr LR] [--model_type [MODEL_TYPE]]
                             [--adj_type [ADJ_TYPE]] [--gpu_id GPU_ID]
                             [--node_dropout_flag NODE_DROPOUT_FLAG]
                             [--node_dropout [NODE_DROPOUT]]
                             [--mess_dropout [MESS_DROPOUT]] [--Ks [KS]]
                             [--save_flag SAVE_FLAG] [--test_flag [TEST_FLAG]]
                             [--report REPORT]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.l

SystemExit: 2

In [13]:
cores

10

In [14]:
multiprocessing.cpu_count()

20